In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
#Download the data
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

In [4]:
#Configuration
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = 'fra.txt'

In [5]:
#Prepare the data fron the file
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [7]:
print(input_characters)

[' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']


In [8]:
print(target_characters)

['\t', '\n', ' ', '!', '$', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']


In [9]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [11]:
#Creating our model
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [12]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 8s 23ms/step - loss: 1.8684 - accuracy: 0.7014 - val_loss: 1.0934 - val_accuracy: 0.7012
Epoch 2/100
125/125 [==============================] - 2s 14ms/step - loss: 0.9206 - accuracy: 0.7473 - val_loss: 0.9222 - val_accuracy: 0.7523
Epoch 3/100
125/125 [==============================] - 2s 14ms/step - loss: 0.7797 - accuracy: 0.7895 - val_loss: 0.7986 - val_accuracy: 0.7804
Epoch 4/100
125/125 [==============================] - 2s 14ms/step - loss: 0.6738 - accuracy: 0.8124 - val_loss: 0.7144 - val_accuracy: 0.7979
Epoch 5/100
125/125 [==============================] - 2s 15ms/step - loss: 0.6258 - accuracy: 0.8258 - val_loss: 0.8632 - val_accuracy: 0.7673
Epoch 6/100
125/125 [==============================] - 2s 14ms/step - loss: 0.6797 - accuracy: 0.8101 - val_loss: 0.7004 - val_accuracy: 0.7982
Epoch 7/100
125/125 [==============================] - 2s 14ms/step - loss: 0.5928 - accuracy: 0.8281 - val_loss: 0.6645 - val_accuracy:

125/125 [==============================] - 2s 14ms/step - loss: 0.1925 - accuracy: 0.9415 - val_loss: 0.4545 - val_accuracy: 0.8775
Epoch 58/100
125/125 [==============================] - 2s 14ms/step - loss: 0.1861 - accuracy: 0.9438 - val_loss: 0.4610 - val_accuracy: 0.8773
Epoch 59/100
125/125 [==============================] - 2s 14ms/step - loss: 0.1831 - accuracy: 0.9441 - val_loss: 0.4649 - val_accuracy: 0.8762
Epoch 60/100
125/125 [==============================] - 2s 14ms/step - loss: 0.1794 - accuracy: 0.9456 - val_loss: 0.4636 - val_accuracy: 0.8770
Epoch 61/100
125/125 [==============================] - 2s 14ms/step - loss: 0.1758 - accuracy: 0.9473 - val_loss: 0.4656 - val_accuracy: 0.8785
Epoch 62/100
125/125 [==============================] - 2s 18ms/step - loss: 0.1729 - accuracy: 0.9471 - val_loss: 0.4653 - val_accuracy: 0.8776
Epoch 63/100
125/125 [==============================] - 2s 14ms/step - loss: 0.1699 - accuracy: 0.9484 - val_loss: 0.4676 - val_accuracy: 0.878

In [16]:
model = keras.models.load_model("s2s")
encoder_inputs = model.input[0]  # input_1
print(encoder_inputs)
print(model.layers[2])
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

KerasTensor(type_spec=TensorSpec(shape=(None, None, 71), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [23]:

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0
    
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

In [25]:
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(20*"-")
    print("Input sentence:", input_texts[seq_index])
    print("Target sentence:", target_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

--------------------
Input sentence: Go.
Target sentence:  	Va !

Decoded sentence: Bouge !

--------------------
Input sentence: Go.
Target sentence:  	Marche.

Decoded sentence: Bouge !

--------------------
Input sentence: Go.
Target sentence:  	Bouge !

Decoded sentence: Bouge !

--------------------
Input sentence: Hi.
Target sentence:  	Salut !

Decoded sentence: Salut !

--------------------
Input sentence: Hi.
Target sentence:  	Salut.

Decoded sentence: Salut !

--------------------
Input sentence: Run!
Target sentence:  	Cours !

Decoded sentence: File !

--------------------
Input sentence: Run!
Target sentence:  	Courez !

Decoded sentence: File !

--------------------
Input sentence: Run!
Target sentence:  	Prenez vos jambes à vos cous !

Decoded sentence: File !

--------------------
Input sentence: Run!
Target sentence:  	File !

Decoded sentence: File !

--------------------
Input sentence: Run!
Target sentence:  	Filez !

Decoded sentence: File !

--------------------
